In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter, FuncFormatter
import torch, pickle, sys, numpy
sys.path.append('/home/dgotzens/scripts/')
import pdfdefaults as pdf
# setup
#pdf.setup()
nfft = 2**18

pi = 3.141592653589 
maxdist = 50
lightspeed = 299_792_458 
f_start, f_end = 76_009_996_288.0, 80_289_505_280.0
t_chirp = 0.000_064_890_002_249_740_060
hertz_per_meter = (f_end-f_start) / t_chirp / lightspeed
bins_per_meter = nfft / maxdist

folder = '/home/dgotzens/scripts/2.2_antenna-characteristics/measured/'
params = pickle.load(open(folder+'feparams.pkl', 'rb'))
refllocs = pickle.load(open(folder+'refl_params.pkl', 'rb'))
tx,ty,rx,ry = params['txPosX'], params['txPosY'], params['rxPosX'], params['rxPosY']

def r_est(R, theta0, exc, theta, K=len(tx)):
    txpos = torch.tensor([tx,ty,[-exc]*K])
    rxpos = torch.tensor([rx,ry,[-exc]*K])
    reflpos = (R-exc)*torch.stack([-torch.sin(theta-theta0),\
                            torch.zeros(len(theta)), \
                            torch.cos(theta-theta0)],0)

    return 0.5*torch.sqrt(((txpos[:,:,None]-reflpos[:,None,:])**2).sum(0))\
          + 0.5*torch.sqrt(((rxpos[:,:,None]-reflpos[:,None,:])**2).sum(0))

In [2]:
measurement = 'a'
results = []
for n, reflloc in enumerate(refllocs):
    R, theta0, exc = reflloc['dist'], reflloc['angle'], reflloc['excentricity']
    with open(f'{folder}{measurement}{int(R):02d}_angle.pkl', 'rb') as f:
        angle = torch.tensor(pickle.load(f))
    refldist = r_est(R,theta0,exc,angle)
    m_refl = ((refldist-int(R)-0.5)*bins_per_meter).long()
    fft = torch.load(f'{folder}{measurement}{int(R):02d}_bp_fft.pt')
    print(f'loaded data for {int(R)}m. processing...')
    M,K,L = fft.shape

    phase = numpy.empty((K,L))
    phase_peak = numpy.empty((K,L))
    for l in range(L):
        phase[:,l] = fft[m_refl[:,l], range(K), l].angle()
        m_peak = fft[:,:,l].abs().argmax(0)
        phase_peak[:,l] = fft[m_peak,range(K),l].angle()
    fft = None
    results.append((phase,phase_peak))
    print('processing complete.')

loaded data for 2m. processing...
processing complete.
loaded data for 8m. processing...
processing complete.
loaded data for 18m. processing...
processing complete.
loaded data for 32m. processing...
processing complete.


In [3]:
pdf.setup()

def unwrap_zero(x, l_zero):
    x_unwrap = numpy.unwrap(x)
    return x_unwrap - 2*pi * (x_unwrap[...,l_zero,None]//(2*pi))

rmse = lambda x, dim=-1 : numpy.sqrt(numpy.mean(numpy.square(x),dim))

N = 1
fig,axes = plt.subplots(N,2,layout='constrained')
fig.set_size_inches(pdf.a4_textwidth, 0.5*pdf.a4_textwidth)
for n, (reflloc, result) in enumerate(zip(refllocs,results[:N])):
    left,right = axes[n,:] if N>1 else axes
    R, theta0, exc = reflloc['dist'], reflloc['angle'], reflloc['excentricity']
    phase, phase_peak = result
    with open(f'{folder}{measurement}{int(R):02d}_angle.pkl', 'rb') as f:
        angle = torch.tensor(pickle.load(f))
    deg = torch.tensor([180/pi*(a-theta0) for a in angle])
    l_sel = [l for l,d in enumerate(deg) if -20<d<20]
    l_zero = deg[l_sel].abs().argmin()

    refldist = r_est(R,theta0,exc,angle)
    model_phase = unwrap_zero(4*pi*f_start/lightspeed*refldist[:,l_sel], l_zero)

    phi = -(model_phase-unwrap_zero(phase[:,l_sel], l_zero)).mean(1)
    phi_peak = -(model_phase-unwrap_zero(phase_peak[:,l_sel], l_zero)).mean(1)

    k=13

    left.plot(deg[l_sel], (model_phase[k,:]-phi[k])/pi, 'k--', label='estimate')
    left.scatter(deg[l_sel[::10]], unwrap_zero(phase[k,l_sel], l_zero)[::10]/pi, marker='.', label='samples')
    left.set_title('Phase at estimated range\n'+\
                   f'$\\hat \\varphi = {phi[k]*180/pi:.3f}$°, '+\
                   f'RMSE = {rmse((model_phase[k,:]+phi[k])-unwrap_zero(phase[k,l_sel], l_zero))*180/pi:.2f}°')

    right.plot(deg[l_sel], (model_phase[k,:]-phi_peak[k])/pi, 'k--', label='estimate')
    right.scatter(deg[l_sel[::10]], unwrap_zero(phase_peak[k,l_sel], l_zero)[::10]/pi, marker='.', label='samples')
    right.set_title('Phase at peak range\n'+\
                   f'$\\hat \\varphi = {phi_peak[k]*180/pi:.3f}$°, '+\
                   f'RMSE = {rmse((model_phase[k,:]+phi_peak[k])-unwrap_zero(phase_peak[k,l_sel], l_zero))*180/pi:.2f}°')
    right.legend()

    for a in (left,right):
        a.yaxis.set_major_formatter(EngFormatter('$\\pi$'))
        a.xaxis.set_major_formatter(EngFormatter('°'))
        a.grid()

fig.savefig('/home/dgotzens/thesis/figures/ch12_phase_linreg.pdf')

In [4]:
Ntx,Nrx = 12,16
phimatrix = 180/pi* 2*phi.reshape((Ntx,Nrx))

phitx = phimatrix[:,0] %(2*pi) - pi
phirx = (phimatrix%(2*pi)-pi - phitx[:,None]).mean(0)
phirx[0] = 0

print([f'{float(a)*180/pi:.3f}' for a in phitx])
print([f'{float(a)*180/pi:.3f}' for a in phirx])

print(rmse((model_phase+phi[:,None])-phase[:,l_sel])[13]/pi)
phi_rmse = rmse((model_phase+phi[:,None])-unwrap_zero(phase[k,l_sel], l_zero)).reshape((Ntx,Nrx))
pdf.setup()
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, (left,right) = plt.subplots(1,2,layout='tight')
fig.set_size_inches(pdf.a4_textwidth,0.4*pdf.a4_textwidth)
im1 = left.pcolormesh(phimatrix/pi)
left.set_yticks(range(0,Ntx+1,3))
left.set_xticks(range(0,Nrx+1,4))
left.set_ylabel('tx-id')
left.set_xlabel('rx-id')
left.set_title('Extrapolated Channel Phase')
left.set_aspect('equal')
left.grid()

divider = make_axes_locatable(left)
cax = divider.append_axes('right', size='5%', pad=0.1)
fig.colorbar(im1 , cax=cax, orientation='vertical', format=EngFormatter('°'))

im2 = right.pcolormesh(phi_rmse/pi)
right.grid()
right.set_yticks(range(0,Ntx+1,3))
right.set_xticks(range(0,Nrx+1,4))
right.set_ylabel('tx-id')
right.set_xlabel('rx-id')
right.set_title('RMSE')
right.set_aspect('equal')

divider = make_axes_locatable(right)
cax = divider.append_axes('right', size='5%', pad=0.1)
fig.colorbar(im2, cax=cax, orientation='vertical', format=FuncFormatter(lambda x,pos: f'{x:3g}$\\pi$'))
fig.savefig('/home/dgotzens/thesis/figures/phase_linreg.pdf')

['-72.807', '71.611', '-83.291', '-73.113', '152.049', '-42.869', '16.291', '-14.481', '58.644', '-177.029', '-164.594', '72.308']
['0.000', '75.810', '37.809', '31.423', '27.896', '23.345', '24.635', '24.994', '68.985', '24.603', '49.326', '8.750', '-13.038', '-22.578', '-3.031', '19.782']
7.390637717992211
